In [3]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RewardLM.git
# %cd RewardLM/
# !pip install -r requirements.txt
# from huggingface_hub import login
# login(token = '')  # https://huggingface.co/settings/tokens

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /Users/daniel/.cache/huggingface/token
Login successful


# ⚖️ ToxicityMeter

In [ ]:
from transformers import GenerationConfig
from rewardlm.ToxicityMeter import ToxicityMeter
from rewardlm.utils.general_utils import device_selector

model_id = 'bigscience/bloomz-3b'
reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target'
batchsize = 16
generation_config = GenerationConfig(
    max_new_tokens = 25,
    num_beams = 5,
    early_stopping = True,
    # crashes while using batchsize > 1 on mps device if not set:
    pad_token_id = 0,
    temperature = 0.8,
    top_p = .8,
    # diversity_penalty = .1, # should use num_beam_groups > 1
)

load_dtype = 'bf16' # can be ['8-bit', 'bf16', 'fp32']
toxicity_meter = ToxicityMeter(
    model_id = model_id,
    load_dtype = load_dtype,
    toxicity_model_id = reward_model_id,
    device = device_selector(),
    generation_config = generation_config,
)

In [ ]:
from rewardlm.data.data_utils import get_real_toxicity_prompts

custom_prompt = ('Statement: "{prompt}".\nResponse: ')
# custom prompts required by the original paper of RedPajama
if model_id == 'togethercomputer/RedPajama-INCITE-Chat-3B-v1':
    custom_prompt = ('<human>: "{prompt}"\n<bot>: ')

toxicity_df = toxicity_meter.measure_toxicity(
    text_prompt = get_real_toxicity_prompts()['text'].to_list(),
    custom_prompt = custom_prompt, 
    generation_config = generation_config,
    batch_size = batchsize,
)

In [ ]:
# save csv in tmp folder
fld = './result analysis/tmp'
toxicity_df.to_csv(
    fld + f'/measured_tox_instruct_{model_id.split("/")[-1]}_{load_dtype}.csv'
)

# 🥞 Reinforcement Learning with Automatic Feedback (RLAF)

In [ ]:
import torch
from rewardlm.core.RL.RLModel import RLModel
from rewardlm.data.data_utils import get_DIALOCONAN_prepro

rlmanager = RLModel(
    model_id = 'EleutherAI/pythia-70m',
    reward_model_id = 'facebook/roberta-hate-speech-dynabench-r4-target',
    optimized = True,   # use 8-bit PEFT
    # log_method = 'wandb',
    bs = 256,
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)


data = get_DIALOCONAN_prepro()
dataset = rlmanager.generate_dataset(
    text = data,
    max_len = 50,
)

history = rlmanager.train_PPO(dataset = dataset)

# 👨🏼‍🏫 Model fine-tune

In [ ]:
import torch
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.data.CustomDatasets import PromptDataset_CLM


# select model
model_id = 'facebook/opt-350m'
generator_manager = GenerativeModel(
    model_id,
    load_dtype = '8-bit' if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

# download dataset
data = get_DIALOCONAN_prepro()

dataset = PromptDataset_CLM(
    tokenizer = generator_manager.tokenizer,
    text = data,
    custom_prompt = custom_prompt,
)

generator_manager.fine_tune(
    torch_dataset = dataset, 
    optimized = True if torch.cuda.is_available() else False,
)